In [1]:
import os
import sys
from pathlib import Path
import argparse
import numpy as np
import torch
import glob
import PIL.Image as pil
import cv2
from  matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from tqdm import tqdm
import os
import sys
sys.path.append('..')

import transformations
basepath = "/home/wuzr/dir/time_func/NewYork_hard"
# mega_path = "Datasets/high-simple"
mega_path = "./datasets/hardnew"
K = np.array([[480,0.000000, 480 ,0.000000],
[0.000000 ,480 ,240 ,0.000000],
[0.000000 ,0.000000 ,1.000000 ,0.000000],
[0.000000 ,0.000000 ,0.000000 ,1.000000]])
K_inv = np.linalg.inv(K)
if not os.path.exists(mega_path):
    os.makedirs(mega_path)
distortion = np.array([0,0,0,0])
pose_scale_factor = 1 #根据下面自动来，把位姿放缩为[-1,1]
num_val = 10



In [7]:
depth_names = [x for x in Path(basepath+"/depth_aligned").iterdir() if x.suffix == '.pfm']
rgb_names = [x for x in Path(basepath+"/rgb").iterdir() if x.suffix == '.jpg']
rgb_names = sorted(rgb_names, key= lambda x:x.stem)
depth_names = sorted(depth_names, key= lambda x:x.stem)
print(len(rgb_names), len(depth_names))

84 84


In [14]:
mask = np.zeros(len(rgb_names))
for i,n in enumerate(depth_names):
    for i2, n2 in enumerate(rgb_names):
        if n2.stem == n.stem:
            mask[i2] = 1
            break

In [20]:
if mask.sum() == len(depth_names):
    rgb_names_new = np.array(rgb_names)[np.arange(len(rgb_names))[mask.astype(np.bool_)]]
origin_sample = np.arange(len(rgb_names))
depth_names_down = np.array(depth_names)[origin_sample]
rgb_names_new_down = np.array(rgb_names_new)[origin_sample]
# poses_down = np.loadtxt(basepath+"/hard.txt").reshape(-1,4,4)[origin_sample]

poses_down = np.zeros((len(rgb_names_new_down), 4, 4))
for i, rgb_name in enumerate(rgb_names_new_down):
    pose_filename = rgb_name.parent/ f'{rgb_name.stem}.txt'
    pose = np.loadtxt(pose_filename).reshape(4, 4)
    poses_down[i] = pose

In [21]:
depths = depth_names_down
images = rgb_names_new_down

In [22]:
def write_pfm(data, fpath, scale=1, file_identifier=b'Pf', dtype="float32"):
    # PFM format definition: http://netpbm.sourceforge.net/doc/pfm.html

    data = np.flipud(data)
    height, width = np.shape(data)[:2]
    values = np.ndarray.flatten(np.asarray(data, dtype=dtype))
    endianess = data.dtype.byteorder
    # print(endianess)

    if endianess == '<' or (endianess == '=' and sys.byteorder == 'little'):
        scale *= -1
    with open(fpath, 'wb') as file:
        file.write((file_identifier))
        file.write(('\n%d %d\n' % (width, height)).encode())
        file.write(('%d\n' % scale).encode())

        file.write(values)
        

        
def read_pfm(fpath, expected_identifier="Pf"):
    # PFM format definition: http://netpbm.sourceforge.net/doc/pfm.html
    
    def _get_next_line(f):
        next_line = f.readline().decode('utf-8').rstrip()
        # ignore comments
        while next_line.startswith('#'):
            next_line = f.readline().rstrip()
        return next_line
    
    with open(fpath, 'rb') as f:
        #  header
        identifier = _get_next_line(f)
        if identifier != expected_identifier:
            raise Exception('Unknown identifier. Expected: "%s", got: "%s".' % (expected_identifier, identifier))

        try:
            line_dimensions = _get_next_line(f)
            dimensions = line_dimensions.split(' ')
            width = int(dimensions[0].strip())
            height = int(dimensions[1].strip())
        except:
            raise Exception('Could not parse dimensions: "%s". '
                            'Expected "width height", e.g. "512 512".' % line_dimensions)

        try:
            line_scale = _get_next_line(f)
            scale = float(line_scale)
            assert scale != 0
            if scale < 0:
                endianness = "<"
            else:
                endianness = ">"
        except:
            raise Exception('Could not parse max value / endianess information: "%s". '
                            'Should be a non-zero number.' % line_scale)

        try:
            data = np.fromfile(f, "%sf" % endianness)
            data = np.reshape(data, (height, width))
            data = np.flipud(data)
            with np.errstate(invalid="ignore"):
                data *= abs(scale)
        except:
            raise Exception('Invalid binary values. Could not create %dx%d array from input.' % (height, width))

        return data


In [23]:
poses = []
inv_pose = None#应该禁用（逆乘第一个位姿）这个不然全局变成了第一个的局部 这里修改只做平移
c2b = np.array([[0,0,1,0],[1,0,0,0],[0,1,0,0],[0,0,0,1]])
for b2w in poses_down:
    c2w = b2w @ transformations.euler_matrix(0,-1.39626,0) @ c2b  
    # c2w = b2w  @ transformations.euler_matrix(0,-np.pi/2,0) @ c2b  
    if inv_pose is None:
        inv_pose = np.eye(4)
        inv_pose[:3,3] = -c2w[:3,3]
        c2w = inv_pose @ c2w
        c2w_RDF = np.linalg.inv(c2b) @ c2w
    else:
        c2w = inv_pose @ c2w #相当于位值为相对第一帧
        c2w_RDF = np.linalg.inv(c2b) @ c2w
    # c2w = torch.from_numpy(c2w).float()
    # poses += [c2w]
    c2w_RDF = torch.from_numpy(c2w_RDF).float()
    poses += [c2w_RDF]

In [24]:
print(torch.stack(poses)[:,:3,3].min(axis=0),torch.stack(poses)[:,:3,3].max(axis=0))

torch.return_types.min(
values=tensor([-61.1852,  -0.2647,   0.0000]),
indices=tensor([63, 37,  0])) torch.return_types.max(
values=tensor([5.0655e+01, 1.2669e-01, 1.9450e+02]),
indices=tensor([24, 76, 37]))


In [25]:
np.linalg.inv(c2b)[:3,:3] @ np.array([180,1.8,0])

array([  1.8,   0. , 180. ])

In [26]:
c2b[:3,:3] @ np.array([1,2,3])

array([3, 1, 2])

In [27]:
np.linalg.inv(c2b)[:3,:3] @ np.array([1,2,3])

array([2., 3., 1.])

In [28]:
transformations.euler_matrix(0,-np.pi/2,0)[:3,:3] @ c2b[:3,:3] @ np.array([1,2,3]) # 因为这个rpy角是相对b的

array([-2.,  1.,  3.])

In [29]:
RDF_TO_DRB = torch.FloatTensor([[0, 1, 0],
                                [1, 0, 0],
                                [0, 0, -1]])#右下前变为下右后（相机坐标一般都是右下前）
mega_poses = []
for c2w in poses:
    c2w = torch.hstack((
    RDF_TO_DRB @ c2w[:3, :3] @ torch.inverse(RDF_TO_DRB),
    RDF_TO_DRB @ c2w[:3, 3:]))
    mega_poses.append(c2w)
positions = torch.cat([c2w[:3, 3].unsqueeze(0) for c2w in mega_poses])
print('{} images'.format(positions.shape[0]))
max_values = positions.max(0)[0]
min_values = positions.min(0)[0]
origin = ((max_values + min_values) * 0.5)
dist = (positions - origin).norm(dim=-1)
diagonal = dist.max()

84 images


In [30]:
max_values - min_values

tensor([  0.3914, 111.8402, 194.5003])

In [31]:
max_values

tensor([ 0.1267, 50.6550,  0.0000])

In [32]:
min_values

tensor([  -0.2647,  -61.1852, -194.5003])

In [33]:
origin

tensor([-6.8993e-02, -5.2651e+00, -9.7250e+01])

In [34]:
(torch.eye(4)[:3, 3] - origin) / pose_scale_factor

tensor([6.8993e-02, 5.2651e+00, 9.7250e+01])

In [35]:
pose_scale_factor = torch.abs(positions).max() + 1 #除法，放缩几倍
pose_scale_factor = 200 #先写死

In [36]:
torch.abs(positions).max()

tensor(194.5003)

In [37]:
if not os.path.exists(os.path.join(mega_path,"train/metadata")): os.makedirs(os.path.join(mega_path,"train/metadata"))
if not os.path.exists(os.path.join(mega_path,"val/metadata")): os.makedirs(os.path.join(mega_path,"val/metadata"))
if not os.path.exists(os.path.join(mega_path,"train/rgbs")): os.makedirs(os.path.join(mega_path,"train/rgbs"))
if not os.path.exists(os.path.join(mega_path,"val/rgbs")): os.makedirs(os.path.join(mega_path,"val/rgbs"))
if not os.path.exists(os.path.join(mega_path,"train/depth")): os.makedirs(os.path.join(mega_path,"train/depth"))
if not os.path.exists(os.path.join(mega_path,"val/depth")): os.makedirs(os.path.join(mega_path,"val/depth"))
if not os.path.exists(os.path.join(mega_path,"train/depthvis")): os.makedirs(os.path.join(mega_path,"train/depthvis"))
if not os.path.exists(os.path.join(mega_path,"val/depthvis")): os.makedirs(os.path.join(mega_path,"val/depthvis"))

coordinates = {
    'origin_drb': origin,
    'pose_scale_factor': pose_scale_factor
}

In [38]:
with open(os.path.join(mega_path,'mappings.txt'),mode='w') as f:
    for idx, _ in enumerate(tqdm(images)):
        if idx % int(positions.shape[0] / num_val) == 0:
            split_dir = os.path.join(mega_path,"val")
        else:
            split_dir = os.path.join(mega_path,"train")
        color = cv2.imread(str(images[idx]),-1)
        cv2.imwrite(os.path.join(split_dir,'rgbs','{0:06d}.jpg'.format(idx)),color)
        depth = read_pfm(str(depths[idx]))
        # depth = np.flip(depth,axis=0)
        write_pfm(depth,os.path.join(split_dir,'depth','{0:06d}.pfm'.format(idx)))
        depthvis = pil.fromarray(depth).convert('RGB')
        depthvis.save(os.path.join(split_dir,'depthvis','{0:06d}.jpg'.format(idx)))
        

        camera_in_drb = mega_poses[idx].clone()#这个操作会改变原来的
        camera_in_drb[:, 3] = (camera_in_drb[:, 3] - origin) / pose_scale_factor

        assert np.logical_and(camera_in_drb >= -1, camera_in_drb <= 1).all()
        

        metadata_name = '{0:06d}.pt'.format(idx)
        torch.save({
            'H': color.shape[0],
            'W': color.shape[1],
            'c2w': torch.cat(
                [camera_in_drb[:, 1:2], -camera_in_drb[:, :1], camera_in_drb[:, 2:4]],
                -1),#变为右上后，又和nice-slam（nerf-pytorch保持一致了），局部为右上后，全局为下右后（不懂为啥要搞这么复杂）
            'intrinsics': torch.FloatTensor(
                [K[0][0], K[1][1], K[0][2], K[1][2]]),
            'distortion': torch.FloatTensor(distortion),
            'timestamp': torch.tensor(float(images[idx].stem),dtype=torch.float64)
        }, os.path.join(split_dir,'metadata',metadata_name))
        f.write('{},{}\n'.format('{0:06d}.jpg'.format(idx), metadata_name))

torch.save(coordinates, os.path.join(mega_path,'coordinates.pt'))


100%|██████████| 84/84 [00:02<00:00, 36.19it/s]


In [39]:
print(poses_down[:,:3,3].min(axis=0),poses_down[:,:3,3].max(axis=0))

[  5.91883707 -61.18521118 -80.81902313] [200.41912842  50.65497589 -80.42766571]


In [40]:
print(torch.stack(poses)[:,:3,3].min(axis=0),torch.stack(poses)[:,:3,3].max(axis=0))

torch.return_types.min(
values=tensor([-61.1852,  -0.2647,   0.0000]),
indices=tensor([63, 37,  0])) torch.return_types.max(
values=tensor([5.0655e+01, 1.2669e-01, 1.9450e+02]),
indices=tensor([24, 76, 37]))


In [41]:
print(torch.stack(mega_poses)[:,:3,3].min(axis=0),torch.stack(mega_poses)[:,:3,3].max(axis=0))

torch.return_types.min(
values=tensor([  -0.2647,  -61.1852, -194.5003]),
indices=tensor([37, 63, 37])) torch.return_types.max(
values=tensor([ 0.1267, 50.6550,  0.0000]),
indices=tensor([76, 24,  0]))


In [42]:
a = []
for c2w in mega_poses:
    camera_in_drb = c2w.clone()#这个操作会改变原来的
    camera_in_drb[:, 3] = (camera_in_drb[:, 3] - origin) / pose_scale_factor
    a.append(torch.cat([camera_in_drb[:, 1:2], -camera_in_drb[:, :1], camera_in_drb[:, 2:4]],-1))
print(torch.stack(a)[:,:3,3].min(axis=0),torch.stack(a)[:,:3,3].max(axis=0))

torch.return_types.min(
values=tensor([-0.0010, -0.2796, -0.4863]),
indices=tensor([37, 63, 37])) torch.return_types.max(
values=tensor([0.0010, 0.2796, 0.4863]),
indices=tensor([76, 24,  0]))


In [43]:
origin

tensor([-6.8993e-02, -5.2651e+00, -9.7250e+01])